

### Установление закономерностей, влияющих на успех продукта на рынке компьютерных игр
____
##### В данном исследовании представлены графики и таблицы, показывающие состояние рынка компьютерных игр и игровых платформ на момент 2016 года в Северной Америке, Европе и Японии.
____
    
##### Данное исследование можно разделить на несколько частей.
#### Изучение общей информации: 
* [1. Изученеие файлов с данными, получение общей информации, загрузка библиотек](#1-bullet)
* [2. Нахождение и обработка пропусков, приведение данных к нужным типам](#2-bullet) 
* [3. Исследовательский анализ данных](#3-bullet) 
*    [3.1 Как менялись продажи по платформам](#3.1-bullet)
*    [3.2 Корреляции продаж и отзывов ](#3.2-bullet)
* [4. Портрет пользователя каждого региона](#4-bullet) 
* [5. Проверка гипотез ](#5-bullet)
*    [5.1 Средние пользовательские рейтинги платформ Xbox One и PC одинаковые](#5.1-bullet)
*    [5.2 Средние пользовательские рейтинги жанров Action и Sports  разные.](#5.2-bullet)
* [6. Общий вывод ](#6-bullet)   
    

## Описание данных 

- Name — название игры
- Platform — платформа
- Year_of_Release — год выпуска
- Genre — жанр игры
- NA_sales — продажи в Северной Америке (миллионы проданных копий)
- EU_sales — продажи в Европе (миллионы проданных копий)
- JP_sales — продажи в Японии (миллионы проданных копий)
- Other_sales — продажи в других странах (миллионы проданных копий)
- Critic_Score — оценка критиков (максимум 100)
- User_Score — оценка пользователей (максимум 10)
- Rating — рейтинг от организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
df.info()
df.describe()

In [ ]:
df.sample(10)

In [ ]:
df['Rating'].unique()

### Вывод

- Необходимо поменять тип данных в 'Year_of_Realese' 'Critic_ core' на целочисленный, 'User_Score' на численный
- Cуществуют пропуски в столбцах 'Name','Year_of_Release','Genre','Critic_Score','User_Score'
- Данные в датасете адекватны

### Шаг 2. Подготовка данных <a name="2-bullet"></a>

##### Приведение столбцов к нижнему регистру

In [ ]:
df.columns = df.columns.str.lower()

##### замена типов данных

In [ ]:
df.info()

Замена в 'user_score' осуществляется потому что в столбце хранятся не строковые данные.

In [ ]:
df['user_score'] = pd.to_numeric(df['user_score'],errors='coerce')

Замена в 'year_of_release','critic_score' осуществляется потому что в столбцах хранятся целочисленные значения

In [ ]:
df_int=['year_of_release','critic_score']
df[df_int]=df[df_int].fillna(-9999)
for df_int_i in df_int:
    df[df_int_i]=df[df_int_i].astype('int')

In [ ]:
df.info()

###### заполнение пропусков



В столбце 'Rating' существует такой тип, как RP (Рейтинг ожидается), который по сути тоже является NaNом

In [ ]:
df['rating']= df['rating'].fillna('unknown')
df['rating']= df['rating'].replace('RP','unknown')

- В столбцах 'Name','Genre', 'year_realise' могли появиться пропуски из-за технических ошибок
- В столбцах 'critic_score','user_score' из-за недостатка оценок для формирования какого-то среднего значения. Здесь заполнение пропусков нельзя восстановить по имеющимся данным.


###### общая сумма продаж

In [ ]:
df['sum_sales']=df['na_sales']+df['eu_sales']+df['jp_sales']

### Шаг 3. Проведите исследовательский анализ данных <a name="3-bullet"></a>

##### сколько игр выпускалось в разные годы

In [ ]:
pv_year=df.pivot_table(index='year_of_release',values='sum_sales',aggfunc='count')

In [ ]:
pv_year.reset_index(inplace=True)
pv_year.columns=['year_of_release','sum_sales']
pv_year.head(10)

In [ ]:
pv_year.sort_values(by='year_of_release',ascending= False,inplace=True)
pv_year.head(15)

Важны данные последних 15 лет, так как они позволяют отследить динамику взлетов и падений.

#####  как менялись продажи по платформам <a name="3.1-bullet"></a>

In [ ]:
pv_pl_sale=df.pivot_table(index=['platform'],values='sum_sales',aggfunc='sum')
pv_pl_sale.reset_index(inplace=True)

In [ ]:
pv_pl_sale.sort_values(by='sum_sales',ascending= False,inplace=True)

In [ ]:
pv_pl_sale.head(10)

In [ ]:
top_pl=pv_pl_sale.query('sum_sales>200')

In [ ]:
top_pl

In [ ]:
top_pl_sl=df[df.platform.isin(top_pl['platform'])]

In [ ]:
top_pl_sl.head(10)

In [ ]:
dynamics_by_platform = top_pl_sl.pivot_table(index=['year_of_release','platform'], values='sum_sales', aggfunc=sum).fillna(0)

In [ ]:
dynamics_by_platform.head(10)

In [ ]:
dynamics_by_platform.reset_index(inplace=True)

In [ ]:
dynamics_by_platform=dynamics_by_platform.query('year_of_release>=2002')
dynamics_by_platform.head(15)

In [ ]:
column=dynamics_by_platform.columns.values.tolist() 

In [ ]:
import plotly.express as px
fig = px.line(dynamics_by_platform, x = 'year_of_release',color='platform', y = 'sum_sales', title = 'dynamics of changes in sales of different platforms')
fig.show()

Как правило, новые платформы появляются с разницей от года до трех лет. Старые исчезают в течение трех лет. Жизненный цикл платформы составляет от 7 до 10 лет 

Актуальными можно считать последние 4 года, так можно будет оследить спад продаж платформы, рост спроса.

##### Какие платформы лидируют по продажам, растут или падают? 

In [ ]:
df_modern=df.query('year_of_release>=2012')

In [ ]:
df_modern_earn = df_modern.pivot_table(index=['year_of_release','platform'], values='sum_sales', aggfunc=sum).fillna(0)

In [ ]:
df_modern_earn.reset_index(inplace=True)

In [ ]:
df_modern_earn.sort_values(by='year_of_release',ascending= True,inplace=True)

In [ ]:
fig = px.line(df_modern_earn, x = 'year_of_release',color='platform', y = 'sum_sales', title = 'sales by year')
fig.show()

На данный момент лидируют по продажам 'XOne', 'PS4','3DS'. Продажи абсолютно всех платформ идут на спад. Наиболее прибыльными на 17 год можно назвать назвать только 'XOne', 'PS4'.

In [ ]:
fig = px.box(df_modern, x="platform", y="sum_sales",title='average platform sales')
fig.show()


У большинства платформ большой "хвост". Это говорит о том, что рынок у платформ очень нестабильный в плане прибыли, доход от платформ серьезно зависит от игр, в которые на них можно играть.

Самый лучший результат у "X360", у этой платформы высоко расположена медиана и верхние квантили, длиный верхний "хвост". Хорошие результаты у "PS4", "PS3". К наиболее стабильным можно отнести "XOne",'Wii', к наиболее рискованным "3DS","DS","PC","PSV","PSP".

##### корреляция продаж и отзывов для PS4 <a name="3.2-bullet"></a>

In [ ]:
df_PS4=df_modern.query('platform=="PS4"')

In [ ]:
df_PS4.sort_values(by='year_of_release',ascending= False,inplace=True)

In [ ]:
df_PS4_critic.sort_values(by='critic_score',ascending= True,inplace=True)
df_PS4_user.sort_values(by='user_score',ascending= True,inplace=True)

In [ ]:
fig = px.scatter(df_PS4_critic, x="critic_score", y="sum_sales", trendline="ols",title='correlation between critics and sales for PS4')
fig.show()

In [ ]:
print(df_PS4_critic['critic_score'].corr(df_PS4_critic['sum_sales']))

In [ ]:
fig = px.scatter(df_PS4_user, x="user_score", y="sum_sales", trendline="ols",title='correlation of user ratings and sales for PS4')
fig.show()

In [ ]:
print(df_PS4_user['user_score'].corr((df_PS4_user['sum_sales'])))

Для PS4:
- Небольшая корреляция есть между оценкой критиков и продажами: чем оценки выше, тем выше продажи. Однако высокая оценка - не гарантия высоких продаж. Но нет случаев, когда игра с низкой оценкой критиков имела высокие продажи.
- Нет почти  никакой корреляции между мнением пользователей и продажами. Хорошие продажи могут быть и у игры со средними оценками, и высокие баллы - не гарантия продаж.

###### корреляция продаж и оценок для всех остальных платформ

In [ ]:
df_modern['user_score']=df_modern['user_score'].fillna(-9999)
df_other_user=df_modern.query('(platform !="PS4")&(user_score>0)')
df_other_critic=df_modern.query('(platform !="PS4")&(critic_score>0)')

In [ ]:
fig = px.scatter(df_other_critic, x="critic_score", y="sum_sales", trendline="ols",title='correlation of critic ratings and sales for other platforms')
fig.show()

In [ ]:
print(df_other_critic['critic_score'].corr((df_other_critic['sum_sales'])))

In [ ]:
fig = px.scatter(df_other_user, x="user_score", y="sum_sales", trendline="ols",title='correlation of user ratings and sales for other platforms')
fig.show()

In [ ]:
print(df_other_user['user_score'].corr((df_other_user['sum_sales'])))

Выводы во многом повторяют то, что характерно для PS4: 
- Существует очень низкая корреляция между оценкой критиков и продажами. Высокие оценки не гарантируют высокие продажи, лишь дают на них надежду. Игры с низкими оценками однозначно имеют малые сборы. 
- Практически нет корреляции между мнением пользователей и продажами. Игры даже с низкими оценками могут надеяться на неплохие продажи, высокие оценки не гарантируют продажи, но дают на них куда большую надежду, чем низкие оценки.

В обоих случаях такие результаты с связаны с тем, что оценки никак не зависят от популярности игры и, следовательно, продаж

In [ ]:
genre_sale=df_modern.pivot_table(index=['genre','year_of_release'], values='sum_sales', aggfunc='sum').reset_index()
genre_sale.sort_values(by='year_of_release',ascending= False,inplace= True)
genre_sale.columns=['genre','year_of_release','sum_sales']

In [ ]:
genre_sale.head(10)

In [ ]:
fig = px.line(genre_sale, x = 'year_of_release', color = 'genre', y = 'sum_sales', title = 'sales of game genres')
fig.show()

Продажи самых прибыльных жанров идут на спад, что связано с общим падением рынка. 
- Жанрами с высокими продажами можно назвать: 'Action','Shooter','Role-Playing','Sports'
- Жанрами с низкими продажами можно  назвать: 'Strategy','Adventure','Racing','Platform','Misc','Fighting','Simulation','Puzzle'

### Вывод

- По продажам лидируют XOne и PS4, также игры на этих платформах имеют наибольший медианый доход, при этом у них же длинный хвост больших продаж. 
- Если у игр высокий рейтинг критиков - это еще не значит, что она обречена на большие продажи. Скорее, высокие оценки дают надежду на успех. 
- Игры даже со средними пользовательскими оценками могут иметь большие продажи. Высокие оценки также не гарантируют высокие продажи. 
- Наиболее продаваемыми жанрами являются 'Action','Shooter','Role-Playing','Sports'

### Шаг 4. Составление портрета пользователя каждого региона <a name="4-bullet"></a>

###### топ-5 платформ в разных регионах

In [ ]:
sales=['jp_sales','na_sales','eu_sales']
region_sales=df_modern.pivot_table(index= 'platform',values=sales, aggfunc='sum').reset_index()

In [ ]:
for region in sales:
    region_sales.sort_values(by=region,ascending= False,inplace=True)
    print(region)
    display(region_sales)

В Северной Америке и в Европе топ-5 полностью совпадает и включает в себя PS4,PS3,X360,XOne,3DS. Японский рынок ощутимо отличается: 3DS, который в обоих случаях был замыкающим, здесь на 1 месте. Так же на этом рынке, в отличие от других, лидируют PSV и WiiU. На всех рынках есть PS3, PS4. 

Вероятнее всего, западные платформы прежде всего адаптированы под свои рынки, японские - под свои. Поэтому X360 И XOne менее охотно покупается японцами по сравнению с платформами, которые изначально под японский рынок заточены. По этим же причинам в Америке лучше продается XOne.

###### топ-5 жанров в разных регионах

In [ ]:
genre_sales_genre=df_modern.pivot_table(index= 'genre',values=sales, aggfunc='sum').reset_index()
for region in sales:
    genre_sales_genre.sort_values(by=region,ascending= False,inplace=True)
    print('Сортировка по возрастанию для ',region)
    display(genre_sales_genre)

Топ-5 в Европе и Северной Америке почти полностью совпадает. в них есть жанры: 'Acting', 'Shooter','Sports','Role-Playing'. Однако в Европе популяерн жанр 'Racing', в Америке - 'Misc'. В Японии преобладают 'Fighting','Simulation', 'Role-Playing','Acting', 'Misc'.


Абсолютно везде популярен 'Role-Playing','Acting'.

Европа и Северная Америка гораздо ближе друг к другу в культурном плане, поэтому их предпочтения совпадают вплоть до позиций жанров в рейтинге, а в Японии тот же 'Role-Playing' гораздо популярней, чем в других регионах, и больше "своих" любимых жанров.

###### Влияет ли рейтинг ESRB на продажи в отдельном регионе

In [ ]:
rating_sales=df_modern.pivot_table(index= 'rating',values=sales, aggfunc='sum').reset_index()

In [ ]:
for rating in sales:
    rating_sales.sort_values(by=rating,ascending= False,inplace=True)
    print('Сортировка по возрастанию для ',rating)
    display(rating_sales)

Рейтинг влияет на продажи в отдельном регионе. Рейтинг 'М' любят гораздо сильнее в Европе и Северной Америке, чем в Японии.

##### Выводы

- Абсолютно везде распространены такие приставки как PS3, PS4.
- Самыми распространенными жанрами можно назвать Acting, Role-Playing
- Cамым распространенным  является рейтинг - 'E', потому что он "для всех".

- В своих предпочтениях Европа и Северная Амерка гораздо ближе друг к другу, чем Япония. В этих регионах популярны 'XOne','X360', в то время как в Японии  - PSV и WiiU. Жанры 'Sports', 'Shooter' любят больше в Северной Америке и Европе, в Японии большим спросом пользуются 'Fighting', 'Simulation'. Рейтинг игр 'M' гораздо популярней в Северной Америке и Европе, чем в Японии.

### Шаг 5. Проверьте гипотезы <a name="5-bullet"></a>

######  гипотеза: средние пользовательские рейтинги платформ Xbox One и PC одинаковые <a name="5.1-bullet"></a>

- Нулевая гипотеза: средние пользовательские рейтинги равны.
- Альтернативная гипотеза: средние пользовательские рейтинги не равны.

In [ ]:
df_modern['user_score']=df_modern['user_score'].fillna(-1)
df_user=df_modern.query('user_score>=0')

In [ ]:
sample_XOne=df_user.loc[df_modern['platform']=='XOne','user_score']
sample_PC=df_user.loc[df_modern['platform']=='PC','user_score']

In [ ]:
sample_XOne

In [ ]:
from scipy.stats import levene
alpha = .05
p = levene(sample_XOne, sample_PC)
print('p-значение:', "{:.15f}".format(p.pvalue))
if (p.pvalue > alpha):
    print("Различие дисперсии не значимо")
else:
    print("Различие дисперсии значимо") 

Малое p-значение говорит о том, что выборки имеют разуню дисперсию, так что параметр `equal_var` имеет значение False

In [ ]:
from scipy import stats as st
alpha = .05
results = st.ttest_ind(
    sample_XOne, 
    sample_PC,
    equal_var = False)
print('p-значение:', "{:.15f}".format(results.pvalue))

if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу") 

Нулевая гипотеза о равнестве рейтинга не отвергается. Существует очень высокая вероятность в 55%, что разница в данных случайна. Таким образом, можно предположить, что средние пользовательские рейтинги равны, а заявленная гипотеза верна. 

###### гипотеза: средние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») разные <a name="5.2-bullet"></a>

- Нулевая гипотеза: средние пользовательские рейтинги равны.
- Альтернативная гипотеза: средние пользовательские рейтинги не равны.

In [ ]:
sample_Action=df_user.loc[df_modern['genre']=='Action','user_score']
sample_Sport=df_user.loc[df_modern['genre']=='Sports','user_score']

In [ ]:
alpha = .05
p = levene(sample_Action, sample_Sport)
print('p-значение:', "{:.15f}".format(p.pvalue))
if (p.pvalue > alpha):
    print("Различие дисперсии не значимо")
else:
    print("Различие дисперсии значимо") 

Малое p-значение говорит о том, что выборки имеют разную дисперсию, так что параметр `equal_var` имеет значение False

In [ ]:
alpha = .05
results = st.ttest_ind(
    sample_Action, 
    sample_Sport,
    equal_var = False)
print('p-значение:', "{:.15f}".format(results.pvalue))

if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

Нулевая гипотеза о равнестве рейтинга отвергается. Существует высокая вероятность, что разница в данных не случайна. Таким образом можно предположить, что заявленная гипотеза верна. 

В обоих случаях нулевая гипотеза была составлена так, чтобы в ней обязательно было равенство. Альтернативная гипотеза строилась исходя из нулевой. В обоих случаях она является двухсторонней. Также в обоих случаях мы сравниваем среднее двух генеральных совокупностей, так как данные не зависимы, поэтому применяется метод 'st.ttest_ind'. Пороговое значение взято стандартное.

## Вывод

-  Средние пользовательские оценки на платформах XOne и PC скорее всего  не различаются
- В средних пользовательских рейтингах жанров 'Action','Sports', вероятно, есть разница.

###  Шаг 6. Общий вывод <a name="6-bullet"></a>

- Северная Америка и Европа ближе друг к другу в предпочтениях платформ и жанров игр, чем к Японии.
- Несмотря на различия, можно сформировать макисмально универсальную игру: она должна быть сделана для PS4, в жанре 'Acting' или 'Role-Playing' c рейтингом 'E'. 
- Из-за общего спада продаж рисковано ориентироваться на характреные только японскому рынку жанры и платформы 
- Для европейского и американского рынка успешная игра может быть с рейтингом 'M', жанров 'Shooter','Sports'. В Европе может быть еще успешен 'Racing', в Америке - 'Misc'.
- На европейском и американком рынке разумно рекламировать игры на платформе 'XOne'
- Следует обращать большее внимание на игры с высокими оценками критиков, оценки пользователей хуже коррелируют с продажами
- Успех игры зависит от платформ, на которых в нее можно играть, от их популярности. 